In [1]:
from dask.distributed import Client
import dask.dataframe as dd

client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 8.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52593,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:52710,Total threads: 2
Dashboard: http://127.0.0.1:52711/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:52599,


In [2]:
import pandas as pd
import numpy as np
import datetime
import re

In [15]:
raw_reviews = pd.read_csv("small_reviews.csv")
reviews = raw_reviews[['reviewerID','overall', 'helpful', 'reviewTime']]
helpful_series = reviews['helpful'].apply(lambda x: np.array(re.findall(r"\d+", x)).astype(int))
reviews[['helpful', 'unhelpful']] = pd.DataFrame(helpful_series.to_list())
reviews['reviewTime'] = reviews['reviewTime'].apply(lambda x: int(re.search(r"\d{4}", x).group()))
reviews

/var/folders/0_/tww2_mb540ldlyk4js_q8l8r0000gn/T/ipykernel_72366/4261087665.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews[['helpful', 'unhelpful']] = pd.DataFrame(helpful_series.to_list())
/var/folders/0_/tww2_mb540ldlyk4js_q8l8r0000gn/T/ipykernel_72366/4261087665.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews[['helpful', 'unhelpful']] = pd.DataFrame(helpful_series.to_list())
/var/folders/0_/tww2_mb540ldlyk4js_q8l8r0000gn/T/ipykernel_72366/4261087665.py:5: SettingWithCopyWarning: 

,reviewerID,overall,helpful,reviewTime,unhelpful
0,A2T0RJ91B0PQ03,1.0,0,2014,0
1,A3TYW0XA8HSGWB,5.0,0,2014,0
2,A2CME0TQU2IVVB,5.0,1,2012,1
3,A2E5IDLX7R388S,5.0,0,2014,0
4,A3CIEMYUGV6ZMR,5.0,0,2012,0
...,...,...,...,...,...
4994,A2TEP2FMBLHN1N,4.0,0,2013,0
4995,A2YKHYSK0GBKB0,5.0,0,2012,0
4996,A2453UWGUHLRE4,5.0,0,2013,0
4997,A26CZH1CTMS1Q4,5.0,2,2011,3


In [16]:
review_groups = reviews.groupby('reviewerID')

In [17]:
number_products_rated = review_groups.count()['overall']
avg_ratings = review_groups['overall'].mean()
reviewing_since = review_groups['reviewTime'].min()

In [24]:
helpful_reviews = review_groups['helpful'].sum()
total_reviews = review_groups['helpful'].sum() + review_groups['unhelpful'].sum()

In [25]:
users = pd.DataFrame(index = review_groups.groups.keys())
users['number_products_rated'] = number_products_rated
users['avg_ratings'] = avg_ratings
users['reviewing_since'] = reviewing_since
users['helpful_reviews'] = helpful_reviews
users['total_reviews'] = total_reviews
users.reset_index(inplace=True)
users = users.rename(columns = {'index':'reviewerID (PRIMARY KEY)'})
users


,reviewerID (PRIMARY KEY),number_products_rated,avg_ratings,reviewing_since,helpful_reviews,total_reviews
0,A0118023LZWOCM3IU40P,1,5.0,2013,0,0
1,A02665964IE1SK1DP89W,1,5.0,2012,0,0
2,A08456402WB5TP5C7LLQX,1,5.0,2012,1,2
3,A098492715I4ZGRRYN6Z3,1,5.0,2014,0,0
4,A101LF5ZOEYTHN,1,1.0,2008,0,28
...,...,...,...,...,...,...
4983,AZVTP654OV9UP,1,5.0,2010,4,8
4984,AZWBA8ACACBJC,1,5.0,2013,0,0
4985,AZWW1U604W0N,1,5.0,2001,3,6
4986,AZXIVW59NBUC3,1,5.0,2011,0,1
